## Third Party Scraper Tools Test

### General Tools

#### (1) SNScrape

In [ ]:
import snscrape.modules.instagram as sni
import pandas as pd
import time

In [ ]:
users = ["Amsterdam.encampment"]
all_posts = []

for user in users:
    scraper = sni.InstagramUserScraper(user)
    try:
        for i, post in enumerate(scraper.get_items()):
            if i >= 50:
                break
            all_posts.append({
                "platform": "IG",
                "username": post.username,
                "url": post.url,
                "date": post.date,
                "content": post.content,
                "likes": post.likes,
                "comments": post.comments,
                "is_video": post.isVideo
            })
        time.sleep(10)
    except Exception as e:
        print(f"Error scraping {user}: {e}")

df = pd.DataFrame(all_posts)
print(df.head())


Error scraping Amsterdam.encampment: Got status code 429
Empty DataFrame
Columns: []
Index: []


In [ ]:
import snscrape.modules.twitter as sntwitter

username = "elonmusk"

scraper = sntwitter.TwitterUserScraper(username)

tweets = []
for tweet in scraper.get_items():
    tweets.append({
        "date": tweet.date,
        "id": tweet.id,
        "content": tweet.content,
        "retweets": tweet.retweetCount,
        "likes": tweet.likeCount,
        "replies": tweet.replyCount,
    })

print(f"Get {len(tweets)} tweets")
print(tweets[:5])


ERROR:snscrape.base:Error retrieving https://twitter.com/elonmusk: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /elonmusk (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))"))
CRITICAL:snscrape.base:4 requests to https://twitter.com/elonmusk failed, giving up.
CRITICAL:snscrape.base:Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /elonmusk (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))")), SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /elonmusk (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local i

ScraperException: 4 requests to https://twitter.com/elonmusk failed, giving up.

#### (2) Scrape-up: Selenium

In [1]:
#pip install selenium

In [1]:
#pip install webdriver_manager

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service

users = ["Middleeasteye",
         "Letstalkpalestine",
         "Nationalsjp"]

class User:
    def __init__(self, username: str):
        self.username = username
    
    def user_details(self):
        driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))
        
        try:
            driver.get(f"https://www.instagram.com/{self.username}/")
            
            #Processing Cookies
            try:
                cookie_accept_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[text()='Allow all cookies']"))
                )
                cookie_accept_button.click()
                print("Cookies agreed")
            except Exception:
                print("No cookies options.")
            
            wait = WebDriverWait(driver, 180)
            account_details = wait.until(
                EC.presence_of_all_elements_located(
                    (By.XPATH, '//span[@class="_ac2a"]')
                )
            )
            return {
                "Username": self.username,
                "Number of Posts": account_details[0].text,
                "Number of Followers": account_details[1].text,
                "Number of Following": account_details[2].text,
            }

        except Exception as e:
            message = f"{self.username} not found or an error occurred: {e}"
            return {"Username": self.username, "data": None, "message": message}
        finally:
            driver.quit()
            
for username in users:
    user = User(username=username)
    user_data = user.user_details()
    
    print("--- User Data ---")
    print(user_data)
    print("-----------------")

Cookies agreed
--- User Data ---
{'Username': 'Middleeasteye', 'data': None, 'message': 'Middleeasteye not found or an error occurred: Message: Browsing context has been discarded\nStacktrace:\nRemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8\nWebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:199:5\nNoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:747:5\nassert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:559:13\nassert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4\nGeckoDriver.prototype.findElements@chrome://remote/content/marionette/driver.sys.mjs:1822:15\ndespatch@chrome://remote/content/marionette/server.sys.mjs:318:40\nexecute@chrome://remote/content/marionette/server.sys.mjs:289:16\nonPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20\nonPacket@chrome://remote/content/marionette/server.sys.mjs:263:9\n_onJSONObjectReady/<@chrome://remote/content/marionette/transpor

KeyboardInterrupt: 

### IG

#### (1) Scrapefly

In [3]:
import json
import httpx
import time
import pandas as pd

In [4]:
client = httpx.Client(
    headers={
        # this is internal ID of an instegram backend app. It doesn't change often.
        "x-ig-app-id": "936619743392459",
        # use browser-like features
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9,ru;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept": "*/*",
    }
)

In [5]:
def scrape_user(username: str):
    try:
        response = client.get(
            f"https://i.instagram.com/api/v1/users/web_profile_info/?username={username}",
            timeout=10,
        )
        response.raise_for_status()
        data = response.json()

        if "data" not in data or "user" not in data["data"]:
            raise ValueError(f"No user data found in the response for '{username}'")

        return data["data"]["user"]

    except httpx.HTTPStatusError as e:
        print(f"HTTP error for user '{username}': {e.response.status_code}")
        return None
    except httpx.RequestError as e:
        print(f"Request error for user '{username}': {e}")
        return None
    except json.JSONDecodeError:
        print(f"JSON decode error for user '{username}': Invalid JSON response.")
        return None
    except ValueError as e:
        print(f"Data error for user '{username}': {e}")
        return None

In [6]:
IG_users = ["Middleeasteye",
         "Letstalkpalestine",
         "Nationalsjp"]

user_data_list = []

for user in IG_users:
    print(f"Attempting to scrape user: {user}...")
    user_data = scrape_user(user)

    if user_data:
        user_info = {
            'username': user_data.get('username'),
            'full_name': user_data.get('full_name'),
            'followers': user_data.get('edge_followed_by', {}).get('count'),
            'following': user_data.get('edge_follow', {}).get('count'),
            'posts': user_data.get('edge_owner_to_timeline_media', {}).get('count'),
            'is_private': user_data.get('is_private'),
            'is_verified': user_data.get('is_verified')
        }
        user_data_list.append(user_info)

    time.sleep(5)

df = pd.DataFrame(user_data_list)

print("\nDataFrame created successfully:")
print(df.head())

Attempting to scrape user: Middleeasteye...
Attempting to scrape user: Letstalkpalestine...
Attempting to scrape user: Nationalsjp...

DataFrame created successfully:
            username             full_name  followers  following  posts  \
0      middleeasteye       Middle East Eye    2230378         65  25139   
1  letstalkpalestine  Let’s Talk Palestine     994244        841    222   
2        nationalsjp          National SJP      76810        895   2184   

   is_private  is_verified  
0       False         True  
1       False        False  
2       False        False  


#### (2) Instaloader

In [9]:
#pip install instaloader

In [7]:
import instaloader
import sys

In [8]:
IG_users = ["Middleeasteye",
         "Letstalkpalestine",
         "Nationalsjp"]

L = instaloader.Instaloader()

# Loop through each username in the list
for username in IG_users:
  try:
    # Get the Profile object for the current username
    profile = instaloader.Profile.from_username(L.context, username)

    print(f"Downloading data for user {profile.username}...")

    # Iterate through all posts of the profile and download each one
    for post in profile.get_posts():
        L.download_post(post, username)

    print(f"Data download for {profile.username} is complete!")

  except Exception as e:
    # Print an error message but continue to the next user
    print(f"Error while processing user {username}: {e}")


JSON Query to graphql/query: 403 Forbidden when accessing https://www.instagram.com/graphql/query [retrying; skip with ^C]
JSON Query to graphql/query: 401 Unauthorized - "fail" status, message "Please wait a few minutes before you try again." when accessing https://www.instagram.com/graphql/query?variables=%7B%22data%22%3A%7B%22count%22%3A12%2C%22include_relationship_info%22%3Atrue%2C%22latest_besties_reel_media%22%3Atrue%2C%22latest_reel_media%22%3Atrue%7D%2C%22username%22%3A%22middleeasteye%22%2C%22__relay_internal__pv__PolarisFeedShareMenurelayprovider%22%3Afalse%7D&doc_id=7898261790222653&server_timestamps=true [retrying; skip with ^C]


Error while processing user Middleeasteye: JSON Query to graphql/query: 401 Unauthorized - "fail" status, message "Please wait a few minutes before you try again." when accessing https://www.instagram.com/graphql/query?variables=%7B%22data%22%3A%7B%22count%22%3A12%2C%22include_relationship_info%22%3Atrue%2C%22latest_besties_reel_media%22%3Atrue%2C%22latest_reel_media%22%3Atrue%7D%2C%22username%22%3A%22middleeasteye%22%2C%22__relay_internal__pv__PolarisFeedShareMenurelayprovider%22%3Afalse%7D&doc_id=7898261790222653&server_timestamps=true


JSON Query to graphql/query: 403 Forbidden when accessing https://www.instagram.com/graphql/query [retrying; skip with ^C]
JSON Query to graphql/query: 401 Unauthorized - "fail" status, message "Please wait a few minutes before you try again." when accessing https://www.instagram.com/graphql/query?variables=%7B%22data%22%3A%7B%22count%22%3A12%2C%22include_relationship_info%22%3Atrue%2C%22latest_besties_reel_media%22%3Atrue%2C%22latest_reel_media%22%3Atrue%7D%2C%22username%22%3A%22letstalkpalestine%22%2C%22__relay_internal__pv__PolarisFeedShareMenurelayprovider%22%3Afalse%7D&doc_id=7898261790222653&server_timestamps=true [retrying; skip with ^C]


Error while processing user Letstalkpalestine: JSON Query to graphql/query: 401 Unauthorized - "fail" status, message "Please wait a few minutes before you try again." when accessing https://www.instagram.com/graphql/query?variables=%7B%22data%22%3A%7B%22count%22%3A12%2C%22include_relationship_info%22%3Atrue%2C%22latest_besties_reel_media%22%3Atrue%2C%22latest_reel_media%22%3Atrue%7D%2C%22username%22%3A%22letstalkpalestine%22%2C%22__relay_internal__pv__PolarisFeedShareMenurelayprovider%22%3Afalse%7D&doc_id=7898261790222653&server_timestamps=true


JSON Query to graphql/query: 403 Forbidden when accessing https://www.instagram.com/graphql/query [retrying; skip with ^C]
JSON Query to graphql/query: 401 Unauthorized - "fail" status, message "Please wait a few minutes before you try again." when accessing https://www.instagram.com/graphql/query?variables=%7B%22data%22%3A%7B%22count%22%3A12%2C%22include_relationship_info%22%3Atrue%2C%22latest_besties_reel_media%22%3Atrue%2C%22latest_reel_media%22%3Atrue%7D%2C%22username%22%3A%22nationalsjp%22%2C%22__relay_internal__pv__PolarisFeedShareMenurelayprovider%22%3Afalse%7D&doc_id=7898261790222653&server_timestamps=true [retrying; skip with ^C]


Error while processing user Nationalsjp: JSON Query to graphql/query: 401 Unauthorized - "fail" status, message "Please wait a few minutes before you try again." when accessing https://www.instagram.com/graphql/query?variables=%7B%22data%22%3A%7B%22count%22%3A12%2C%22include_relationship_info%22%3Atrue%2C%22latest_besties_reel_media%22%3Atrue%2C%22latest_reel_media%22%3Atrue%7D%2C%22username%22%3A%22nationalsjp%22%2C%22__relay_internal__pv__PolarisFeedShareMenurelayprovider%22%3Afalse%7D&doc_id=7898261790222653&server_timestamps=true


In [13]:
def check_instagram_account_status(username):
    """
    Checks the status of an Instagram account.
    """
    L = instaloader.Instaloader()
    try:
        # Get the Profile object from the username
        profile = instaloader.Profile.from_username(L.context, username)

        # Check if the account is private
        if profile.is_private:
            print(f"Account @{username} is private.")
            return "Private"

        # Check if the account is verified
        if profile.is_verified:
            print(f"Account @{username} is verified.")
        
        # If no exceptions, the account is public and accessible
        print(f"Account @{username} is public and accessible via the API.")
        return "Public and Accessible"

    except instaloader.exceptions.ProfileNotExistsException:
        # This exception is raised if the account does not exist or is suspended
        print(f"Account @{username} does not exist or is suspended.")
        return "Not Exists or Suspended"
    except Exception as e:
        # Catch any other unknown errors
        print(f"An unknown error occurred while checking @{username}: {e}")
        return "Error"

# List of Instagram usernames to check
IG_users = ["letstalkpalestine", "nationalsjp", "this_user_does_not_exist_123", "this_user_does_not_exist_123456"]

for user in IG_users:
    status = check_instagram_account_status(user)
    print(f"Status for @{user}: {status}\n")

Account @letstalkpalestine is public and accessible via the API.
Status for @letstalkpalestine: Public and Accessible

Account @nationalsjp is public and accessible via the API.
Status for @nationalsjp: Public and Accessible

Account @this_user_does_not_exist_123 is private.
Status for @this_user_does_not_exist_123: Private



JSON Query to api/v1/users/web_profile_info/?username=this_user_does_not_exist_123456: 404 Not Found when accessing https://i.instagram.com/api/v1/users/web_profile_info/?username=this_user_does_not_exist_123456 [retrying; skip with ^C]
JSON Query to api/v1/users/web_profile_info/?username=this_user_does_not_exist_123456: 404 Not Found when accessing https://i.instagram.com/api/v1/users/web_profile_info/?username=this_user_does_not_exist_123456 [retrying; skip with ^C]
JSON Query to web/search/topsearch/: 401 Unauthorized - "fail" status, message "Server Error" when accessing https://www.instagram.com/web/search/topsearch/?context=blended&query=this_user_does_not_exist_123456&include_reel=False&__a=1 [retrying; skip with ^C]
JSON Query to web/search/topsearch/: 401 Unauthorized - "fail" status, message "Server Error" when accessing https://www.instagram.com/web/search/topsearch/?context=blended&query=this_user_does_not_exist_123456&include_reel=False&__a=1 [retrying; skip with ^C]


An unknown error occurred while checking @this_user_does_not_exist_123456: JSON Query to web/search/topsearch/: 401 Unauthorized - "fail" status, message "Server Error" when accessing https://www.instagram.com/web/search/topsearch/?context=blended&query=this_user_does_not_exist_123456&include_reel=False&__a=1
Status for @this_user_does_not_exist_123456: Error



### Twitter

#### （1）Twikit

In [6]:
#pip install twikit

In [14]:
import asyncio
from twikit.guest import GuestClient

In [23]:
client = GuestClient()

async def main():
    # Activate the client by generating a guest token.
    await client.activate()

    # Get user by screen name
    user = await client.get_user_by_screen_name('elonmusk')
    
    print(f"--- Getting latest 10 tweets from @{user.name} ---")
    
    # Get the latest 10 tweets from the user
    user_tweets = await client.get_user_tweets(user.id, count=10)
    
    if user_tweets:
        print(f"Successfully retrieved {len(user_tweets)} tweets.")
        print("-" * 30)
        
        # Iterate through the list of tweets and print details
        for i, tweet in enumerate(user_tweets):
            print(f"Tweet {i+1} (ID: {tweet.id})")
            print(f"  Text: {tweet.text}")
            print(f"  Likes: {tweet.favorite_count}")
            print("-" * 30)
    else:
        print(f"Could not find any tweets for @{user.name}.")

# In a Jupyter/IPython environment, you can use 'await main()'
# If running as a script, use asyncio.run(main())
try:
    asyncio.run(main())
except RuntimeError:
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())

--- Getting latest 10 tweets from @Elon Musk ---
Successfully retrieved 98 tweets.
------------------------------
Tweet 1 (ID: 1519480761749016577)
  Text: Next I’m buying Coca-Cola to put the cocaine back in
  Likes: 4319243
------------------------------
Tweet 2 (ID: 1812258574049157405)
  Text: https://t.co/6eOgN9UdOy
  Likes: 3329509
------------------------------
Tweet 3 (ID: 1518623997054918657)
  Text: I hope that even my worst critics remain on Twitter, because that is what free speech means
  Likes: 2936447
------------------------------
Tweet 4 (ID: 1854026234339938528)
  Text: 🇺🇸🇺🇸The future is gonna be so 🔥 🇺🇸🇺🇸 https://t.co/x56cqb6oT5
  Likes: 2607387
------------------------------
Tweet 5 (ID: 1519495072802390016)
  Text: Let’s make Twitter maximum fun!
  Likes: 2401069
------------------------------
Tweet 6 (ID: 1518677066325053441)
  Text: 🚀💫♥️ Yesss!!! ♥️💫🚀 https://t.co/0T9HzUHuh6
  Likes: 2370545
------------------------------
Tweet 7 (ID: 1519495982723084290)
  Text:

In [21]:
async def check_x_account_status(username):
    """
    Checks the status of an X (Twitter) account.
    """
    try:
        # Activate the client by generating a guest token.
        await client.activate()

        # Get the user object by username.
        user = await client.get_user_by_screen_name(username)

        if user:
            print(f"Account @{username} exists.")
            # Check if the account is protected/private.
            if user.protected:
                print(f"Account @{username} is protected (private).")
                return "Protected"
            
            # Check if the account is accessible.
            print(f"Account @{username} is public and accessible via the API.")
            
            # Simulate search to check if it appears in autocomplete.
            # A successful user object retrieval confirms it's searchable.
            print(f"Account @{username} appears in search results.")
            return "Accessible and Public"
        else:
            print(f"Account @{username} does not exist or is suspended.")
            return "Not Exists or Suspended"

    except Exception as e:
        print(f"An error occurred while checking @{username}: {e}")
        return "Error"

async def main():
    users_to_check = [
        "elonmusk", 
        "realDonaldTrump", 
        "NASA", 
        "Twitter",
        "this_account_does_not_exist_xyz_123"
    ]
    
    for user in users_to_check:
        status = await check_x_account_status(user)
        print(f"Status for @{user}: {status}\n")

try:
    asyncio.run(main())
except RuntimeError:
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())

An error occurred while checking @elonmusk: status: 404, message: ""
Status for @elonmusk: Error

An error occurred while checking @realDonaldTrump: status: 404, message: ""
Status for @realDonaldTrump: Error

An error occurred while checking @NASA: status: 404, message: ""
Status for @NASA: Error

An error occurred while checking @Twitter: status: 404, message: ""
Status for @Twitter: Error

An error occurred while checking @this_account_does_not_exist_xyz_123: status: 404, message: ""
Status for @this_account_does_not_exist_xyz_123: Error

